In [1]:
from sys import modules

IN_COLAB = 'google.colab' in modules
if IN_COLAB:
    !pip install -q ir_axioms python-terrier

In [2]:
# Start/initialize PyTerrier.
from pyterrier import started, init

if not started():
    init(tqdm="notebook")

PyTerrier 0.8.0 has loaded Terrier 5.6 (built by craigmacdonald on 2021-09-17 13:27)

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


In [3]:
from pathlib import Path
from typing import List
from pyterrier import Transformer
from examples.trec_util import TrecTrack

# track = TrecTrack(28, "deep", "msmarco-passage/trec-dl-2019")
track = TrecTrack(
    28, "deep", "msmarco-passage/trec-dl-2019",
    base_results_dir=Path("/home/heinrich/Development/webis/trec-system-runs")
)
retrieval_systems: List[Transformer] = [result % 10 for result in track.results]

Read results: 100%|██████████| 37/37 [00:17<00:00,  2.14run/s]


In [4]:
from pyterrier.pipelines import Experiment
from ir_measures import nDCG

Experiment(
    retr_systems=retrieval_systems,
    topics=track.dataset.get_topics(),
    qrels=track.dataset.get_qrels(),
    eval_metrics=[nDCG @ 10],
    verbose=True,
)

pt.Experiment:   0%|          | 0/37 [00:00<?, ?system/s]

,name,nDCG@10
0,"RankCutoff(Transformer(), 10)",0.467633
1,"RankCutoff(Transformer(), 10)",0.638501
2,"RankCutoff(Transformer(), 10)",0.675600
3,"RankCutoff(Transformer(), 10)",0.494324
4,"RankCutoff(Transformer(), 10)",0.464744
5,"RankCutoff(Transformer(), 10)",0.715782
6,"RankCutoff(Transformer(), 10)",0.522749
7,"RankCutoff(Transformer(), 10)",0.566855
8,"RankCutoff(Transformer(), 10)",0.698716
9,"RankCutoff(Transformer(), 10)",0.585584


In [5]:
from ir_axioms.backend.pyterrier.experiment import AxiomaticExperiment
from ir_axioms.axiom import TFC1, LNC1

axiomatic_experiment = AxiomaticExperiment(
    retrieval_systems=retrieval_systems,
    topics=track.dataset.get_topics(),
    qrels=track.dataset.get_qrels(),
    index=track.index,
    dataset=track.ir_dataset,
    axioms=[TFC1(), LNC1()],
    verbose=True,
)

In [6]:
preferences = axiomatic_experiment.preferences
preferences

AxiomaticExperiment: 100%|██████████| 37/37 [03:40<00:00,  5.95s/system]


,qid,docno_a,rank_a,score_a,name_a,query,docno_b,rank_b,score_b,name_b,original_preference,oracle_preference,TFC1_preference,LNC1_preference
0,1108939,7651010,1,5.532341,srchvrs_ps_run1,what slows down the flow of blood,7651010,1,5.532341,srchvrs_ps_run1,0,NaN,0,0
1,1108939,7651010,1,5.532341,srchvrs_ps_run1,what slows down the flow of blood,5291621,2,5.523702,srchvrs_ps_run1,1,NaN,0,0
2,1108939,7651010,1,5.532341,srchvrs_ps_run1,what slows down the flow of blood,7724054,3,5.452711,srchvrs_ps_run1,1,NaN,0,0
3,1108939,7651010,1,5.532341,srchvrs_ps_run1,what slows down the flow of blood,5163856,4,5.348308,srchvrs_ps_run1,1,NaN,0,0
4,1108939,7651010,1,5.532341,srchvrs_ps_run1,what slows down the flow of blood,2846462,5,5.342947,srchvrs_ps_run1,1,NaN,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15984,532603,7474474,9,0.427272,runid2,university of dubuque enrollment,1204838,5,0.427785,runid2,-1,NaN,0,0
15985,532603,7474474,9,0.427272,runid2,university of dubuque enrollment,5346281,6,0.427509,runid2,-1,NaN,0,0
15986,532603,7474474,9,0.427272,runid2,university of dubuque enrollment,2195902,7,0.427481,runid2,-1,NaN,0,0
15987,532603,7474474,9,0.427272,runid2,university of dubuque enrollment,3920188,8,0.427431,runid2,-1,NaN,0,0


In [7]:
# Judged preferences
judged_preferences = preferences[preferences["oracle_preference"].notna()]
judged_preferences

,qid,docno_a,rank_a,score_a,name_a,query,docno_b,rank_b,score_b,name_b,original_preference,oracle_preference,TFC1_preference,LNC1_preference
1548,156493,2259183,1,6.572989,srchvrs_ps_run1,do goldfish grow,2259183,1,6.572989,srchvrs_ps_run1,0,0.0,0,0
1549,156493,2259183,1,6.572989,srchvrs_ps_run1,do goldfish grow,3288597,2,6.525999,srchvrs_ps_run1,1,0.0,0,0
1550,156493,2259183,1,6.572989,srchvrs_ps_run1,do goldfish grow,3288601,3,6.517504,srchvrs_ps_run1,1,0.0,0,0
1551,156493,2259183,1,6.572989,srchvrs_ps_run1,do goldfish grow,3288596,4,6.508273,srchvrs_ps_run1,1,0.0,0,-1
1552,156493,2259183,1,6.572989,srchvrs_ps_run1,do goldfish grow,3288600,5,6.508273,srchvrs_ps_run1,1,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15660,146187,1230566,9,0.427014,runid2,difference between a mcdouble and a double che...,6430258,5,0.430223,runid2,-1,0.0,-1,0
15661,146187,1230566,9,0.427014,runid2,difference between a mcdouble and a double che...,6658185,6,0.428952,runid2,-1,1.0,0,0
15662,146187,1230566,9,0.427014,runid2,difference between a mcdouble and a double che...,7096817,7,0.427588,runid2,-1,1.0,0,0
15663,146187,1230566,9,0.427014,runid2,difference between a mcdouble and a double che...,1138801,8,0.427014,runid2,-1,1.0,0,0


In [8]:
wrong_preferences = judged_preferences[
    abs(judged_preferences["oracle_preference"] - judged_preferences["original_preference"]) >= 2
    ]
wrong_preferences

,qid,docno_a,rank_a,score_a,name_a,query,docno_b,rank_b,score_b,name_b,original_preference,oracle_preference,TFC1_preference,LNC1_preference
1556,156493,2259183,1,6.572989,srchvrs_ps_run1,do goldfish grow,6139386,9,6.399120,srchvrs_ps_run1,1,-1.0,0,0
1565,156493,3288597,2,6.525999,srchvrs_ps_run1,do goldfish grow,6139386,9,6.399120,srchvrs_ps_run1,1,-1.0,0,0
1574,156493,3288601,3,6.517504,srchvrs_ps_run1,do goldfish grow,6139386,9,6.399120,srchvrs_ps_run1,1,-1.0,0,0
1583,156493,3288596,4,6.508273,srchvrs_ps_run1,do goldfish grow,6139386,9,6.399120,srchvrs_ps_run1,1,-1.0,0,0
1592,156493,3288600,5,6.508273,srchvrs_ps_run1,do goldfish grow,6139386,9,6.399120,srchvrs_ps_run1,1,-1.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15658,146187,1230566,9,0.427014,runid2,difference between a mcdouble and a double che...,2040185,3,0.430896,runid2,-1,1.0,0,0
15659,146187,1230566,9,0.427014,runid2,difference between a mcdouble and a double che...,2257806,4,0.430273,runid2,-1,1.0,0,0
15661,146187,1230566,9,0.427014,runid2,difference between a mcdouble and a double che...,6658185,6,0.428952,runid2,-1,1.0,0,0
15662,146187,1230566,9,0.427014,runid2,difference between a mcdouble and a double che...,7096817,7,0.427588,runid2,-1,1.0,0,0


In [9]:
wrong_preferences_with_axiom_hints = wrong_preferences[
    (wrong_preferences["TFC1_preference"] != 0) | (wrong_preferences["LNC1_preference"] != 0)
    ]
wrong_preferences_with_axiom_hints

,qid,docno_a,rank_a,score_a,name_a,query,docno_b,rank_b,score_b,name_b,original_preference,oracle_preference,TFC1_preference,LNC1_preference
1723,1110199,398442,2,7.017928,srchvrs_ps_run1,what is wifi vs bluetooth,5218014,5,6.803596,srchvrs_ps_run1,1,-1.0,1,0
1741,1110199,554521,4,6.814814,srchvrs_ps_run1,what is wifi vs bluetooth,5218014,5,6.803596,srchvrs_ps_run1,1,-1.0,0,-1
1747,1110199,5218014,5,6.803596,srchvrs_ps_run1,what is wifi vs bluetooth,398442,2,7.017928,srchvrs_ps_run1,-1,1.0,-1,0
1749,1110199,5218014,5,6.803596,srchvrs_ps_run1,what is wifi vs bluetooth,554521,4,6.814814,srchvrs_ps_run1,-1,1.0,0,1
2685,130510,7125239,1,6.547254,srchvrs_ps_run1,definition declaratory judgment,1494936,4,6.341850,srchvrs_ps_run1,1,-1.0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13839,182539,8757181,5,0.432632,runid2,example of monotonic function,57444,2,0.434976,runid2,-1,1.0,1,0
14858,1113437,4981632,1,0.429929,runid2,what is physical description of spruce,8128794,4,0.427766,runid2,1,-1.0,-1,0
14877,1113437,6021898,3,0.428806,runid2,what is physical description of spruce,1066532,5,0.427582,runid2,1,-1.0,1,0
14882,1113437,8128794,4,0.427766,runid2,what is physical description of spruce,4981632,1,0.429929,runid2,-1,1.0,1,0


In [10]:
from ir_axioms.axiom import TFC1, LNC1, OriginalAxiom

tfc1_axiom = TFC1()  # Query term frequency in documents.
lnc1_axiom = LNC1()  # Similarity between query and document terms.
orig_axiom = OriginalAxiom()  # Similarity between query and document terms.

In [11]:
from ir_axioms.axiom import Axiom

# Define axiom for reranker and permutations.
axiom: Axiom = tfc1_axiom + lnc1_axiom + orig_axiom * 2.0

In [12]:
from ir_axioms.backend.pyterrier.transformers import AxiomaticReranker

reranked_retrieval_systems = [
    retrieval_system >> AxiomaticReranker(
        axiom,
        track.index,
        track.dataset_name,
        verbose=True
    )
    for retrieval_system in retrieval_systems
]

In [13]:
from pyterrier.pipelines import Experiment
from ir_measures import nDCG

# noinspection PyTypeChecker
experiment = Experiment(
    [*retrieval_systems, *reranked_retrieval_systems],
    track.dataset.get_topics(),
    track.dataset.get_qrels(),
    [nDCG @ 10],
)
experiment

AxiomaticReranker: 100%|██████████| 200/200 [00:03<00:00, 63.53query/s]


,name,nDCG@10
0,"RankCutoff(Transformer(), 10)",0.467633
1,"RankCutoff(Transformer(), 10)",0.638501
2,"RankCutoff(Transformer(), 10)",0.675600
3,"RankCutoff(Transformer(), 10)",0.494324
4,"RankCutoff(Transformer(), 10)",0.464744
...,...,...
69,"Compose(RankCutoff(Transformer(), 10), <ir_axi...",0.696790
70,"Compose(RankCutoff(Transformer(), 10), <ir_axi...",0.481071
71,"Compose(RankCutoff(Transformer(), 10), <ir_axi...",0.715965
72,"Compose(RankCutoff(Transformer(), 10), <ir_axi...",0.426686
